In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
import sys
import os

# Add the parent directory to the path (adjust if your structure differs)
project_root = r"D:\Minor Project Ayan"
sys.path.append(project_root)

# Check it's added correctly
print("✅ Path added:", project_root)

# Now import the module
from utils.c3d_feature_extractor import extract_c3d_features


video_path = r"D:\Minor Project Ayan\data\Abuse001_x264.mp4"

# Save as .npy
extract_c3d_features(video_path,
                    save_path=r"D:\Minor Project Ayan\data\C3D_Features\Abuse001_features.npy",
                    use_cuda=True,
                    save_format="npy")

# Save as .mat
extract_c3d_features(video_path,
                    save_path=r"D:\Minor Project Ayan\data\C3D_Features\Abuse001_features.mat",
                    use_cuda=True,
                    save_format="mat")


Feature Extraction

In [ ]:
import os
import torch
import numpy as np
from utils.c3d_feature_extractor import extract_c3d_features

# Path to your dataset
dataset_path = r"D:\Minor Project Ayan\dataset_Extras"  # Update with your dataset path

# Subfolder names for each class
subfolders = ["Stealing","Vandalism"]

# Define the path where the extracted features will be saved
output_path = r"D:\Minor Project Ayan\data\C3D_Features"

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# Process all videos in the dataset
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)
    print(f"Processing videos from '{subfolder}'...")

    # Iterate over all videos in the subfolder
    for video_name in os.listdir(subfolder_path):
        video_path = os.path.join(subfolder_path, video_name)
        torch.cuda.empty_cache()
        # Ensure the file is a video (check file extension)
        if video_path.endswith(('.mp4', '.avi', '.mov')):
            print(f"Processing video: {video_name}")
            
            # Define the path to save extracted features
            save_path = os.path.join(output_path, f"{subfolder}_{video_name.split('.')[0]}_features.npy")
            
            # Extract C3D features and save them
            extract_c3d_features(video_path, save_path=save_path, use_cuda=True, save_format="npy")

print("Feature extraction completed for all videos.")


Preparing Train Data

In [ ]:
import os
import numpy as np

# Directory where extracted features are saved
features_dir = r"D:\Minor Project Ayan\data\C3D_Features"

# Output files for binary classification
features_output = r"D:\Minor Project Ayan\data\X_train.npy"
labels_output = r"D:\Minor Project Ayan\data\y_train.npy"

# Define the class that represents normal activity
normal_class = "Normal"

X_train = []
y_train = []

# Loop over all .npy feature files
for file_name in os.listdir(features_dir):
    if not file_name.endswith("_features.npy"):
        continue

    label_name = file_name.split("_")[0]

    # Binary label assignment
    if label_name == normal_class:
        label = 0  # Normal
    else:
        label = 1  # Anomaly

    feature_path = os.path.join(features_dir, file_name)
    features = np.load(feature_path, allow_pickle=True)
    X_train.append(features)
    y_train.append(label)

# Convert to numpy arrays
X_train = np.array(X_train, dtype=object)  # keep as object if variable length
y_train = np.array(y_train)

# Save them
np.save(features_output, X_train)
np.save(labels_output, y_train)

print(f"[INFO] Saved {len(X_train)} feature arrays and binary labels.")


Training Model

In [ ]:
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

# Load the data
X_train = np.load('D:/Minor Project Ayan/data/X_train.npy', allow_pickle=True)
y_train = np.load('D:/Minor Project Ayan/data/y_train.npy', allow_pickle=True)

# Convert labels to binary: 0 = Stealing, 1 = Vandalism
# (This is already done, so you don't need to modify y_train here if it's already correct)

# Ensure proper dtype
X_train = np.array(X_train.tolist(), dtype=np.float32)  # Convert from object dtype
y_train = np.array(y_train, dtype=np.int64)

print(f"X_train shape: {X_train.shape}, dtype: {X_train.dtype}")
print(f"y_train shape: {y_train.shape}, unique values: {np.unique(y_train)}")

# Define the model
model = Sequential([
    LSTM(128, input_shape=(32, 512), return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary output
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

# Callbacks
checkpoint_dir = "D:/Minor Project Ayan/models/"
os.makedirs(checkpoint_dir, exist_ok=True)

callbacks = [
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model.h5'),  # Save as .h5
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    TensorBoard(
        log_dir=os.path.join(checkpoint_dir, 'logs'),
        histogram_freq=1
    )
]

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=60,
    batch_size=8,
    callbacks=callbacks,
    verbose=1
)

# Save final model as .h5
model.save(os.path.join(checkpoint_dir, 'final_model.h5'))  # Final model as .h5
print("✅ Model training complete and saved.")


Generating Training Results

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.model_selection import train_test_split

# Load your data (assuming X_train and y_train are already prepared)
# Make sure your data is in the correct dtype (float32)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Ensure that X_train_split and X_val_split are of type float32
X_train_split = np.array(X_train_split, dtype=np.float32)
X_val_split = np.array(X_val_split, dtype=np.float32)

# Verify the shape and dtype
print(f"X_train_split shape: {X_train_split.shape}, dtype: {X_train_split.dtype}")
print(f"X_val_split shape: {X_val_split.shape}, dtype: {X_val_split.dtype}")

# Load the trained model
model = load_model('D:/Minor Project Ayan/models/best_model.keras')

# Predict probabilities for validation data
y_pred_prob = model.predict(X_val_split)

# Convert probabilities to class predictions (threshold 0.5)
y_pred = (y_pred_prob > 0.5).astype(int)

# Generate confusion matrix
cm = confusion_matrix(y_val_split, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_val_split, y_pred))

# Calculate AUC - ROC
roc_auc = roc_auc_score(y_val_split, y_pred_prob)
print(f"AUC-ROC Score: {roc_auc}")

# ROC curve
fpr, tpr, _ = roc_curve(y_val_split, y_pred_prob)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_val_split, y_pred_prob)
pr_auc = auc(recall, precision)  # Calculate AUC for Precision-Recall curve
print(f"AUC-PR Score: {pr_auc}")

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, color='green', label=f'AUC = {pr_auc:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

# Box plot for Precision and Recall
plt.figure(figsize=(10, 6))
plt.boxplot([precision, recall], vert=True, patch_artist=True, labels=['Precision', 'Recall'])
plt.title('Precision and Recall Box Plot')
plt.show()
